In [1]:
# SparkContext is already defined as sc
HDFS = 'hdfs://scut0:9000/'

# Extracting the right features from your data

## Introduction to Feature hashing

Feature hashing is a technique to deal with **high-dimensional data** and is often **used with text and categorical datasets** where the features can take on many unique values (often many millions of values)

Up until now, we have often used a simple approach of collecting the distinct feature values and zipping this collection with
a set of indices to create a map of feature value to index. This mapping is then
broadcast (either explicitly in our code or implicitly by Spark) to each worker.

When dealing with huge feature dimensions in the tens of millions or more
that are common when working with text, this approach can be slow and can require
signifcant memory and network resources, both on the Spark master (to collect the
unique values) and workers (to broadcast the resulting mapping to each worker,
which keeps it in memory to allow it to apply the feature encoding to its local piece
of the input data)


Also, building and using 1-of-K feature encoding requires us to keep a mapping of each possible feature value to an index in a vector. Furthermore, the process of creating the mapping itself requires at least one additional pass through the dataset and can be tricky to do in parallel scenarios.


**Feature hashing works by assigning the vector index for a feature based on the value obtained by hashing this feature to a number (usually, an integer value) using a hash function. **

This encoding works the same way as mapping-based encoding, except that we choose a size for our feature vector upfront.

Feature hashing has the advantage that we do not need to build a mapping and keep it in memory. It is also easy to implement, very fast, and can be done online and in real time, thus not requiring a pass through our dataset frst. 

However, there are two important drawbacks

1. As we don't create a mapping of features to index values, we also cannot do
the reverse mapping of feature index to value. This makes it harder to, for
example, determine which features are most informative in our models.

2. As we are restricting the size of our feature vectors, we might experience
hash collisions. This happens when two different features are hashed into
the same index in our feature vector. **Surprisingly, this doesn't seem to have
a severe impact on model performance as long as we choose a reasonable
feature vector dimension relative to the dimension of the input data**


## Extracting the TF-IDF features from the 20 Newsgroups dataset

To illustrate the concepts in this chapter, we will use a well-known text dataset called
20 Newsgroups; this dataset is commonly used for text-classifcation tasks. This is a
collection of newsgroup messages posted across 20 different topics. There are various
forms of data available. For our purposes, we will use the bydate version of the
dataset, which is available at http://qwone.com/~jason/20Newsgroups.

### Exploring the 20 Newsgroups data

In [2]:
textFilePairs = sc.wholeTextFiles(HDFS + '20_newsgroup/*')
print(textFilePairs.count())
print(textFilePairs.first())

19997
(u'hdfs://scut0:9000/20_newsgroup/alt.atheism/49960', u'Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew\nFrom: mathew <mathew@mantis.co.uk>\nNewsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers\nSubject: Alt.Atheism FAQ: Atheist Resources\nSummary: Books, addresses, music -- anything related to atheism\nKeywords: FAQ, atheism, books, music, fiction, addresses, contacts\nMessage-ID: <19930329115719@mantis.co.uk>\nDate: Mon, 29 Mar 1993 11:57:19 GMT\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nFollowup-To: alt.atheism\nDistribution: world\nOrganization: Mantis Consultants, Cambridge. UK.\nApproved: news-answers-request@mit.edu\nSupersedes: <19930301143317@mantis.co.uk>\nLines: 290\n\nArchive-name: a

In [3]:
newsGroup = textFilePairs.map(lambda (path, content) : (path.split('/')[-2], 1)).reduceByKey(lambda a, b: a + b)
print(newsGroup.count())
sortedNewsGroup = newsGroup.sortBy(lambda x:x[1], ascending = False)
for ng in sortedNewsGroup.collect():
    print(ng)

20
(u'sci.crypt', 1000)
(u'comp.sys.mac.hardware', 1000)
(u'sci.med', 1000)
(u'comp.windows.x', 1000)
(u'misc.forsale', 1000)
(u'talk.politics.guns', 1000)
(u'comp.os.ms-windows.misc', 1000)
(u'sci.space', 1000)
(u'rec.sport.baseball', 1000)
(u'rec.motorcycles', 1000)
(u'talk.politics.misc', 1000)
(u'comp.graphics', 1000)
(u'talk.religion.misc', 1000)
(u'talk.politics.mideast', 1000)
(u'comp.sys.ibm.pc.hardware', 1000)
(u'alt.atheism', 1000)
(u'rec.sport.hockey', 1000)
(u'sci.electronics', 1000)
(u'rec.autos', 1000)
(u'soc.religion.christian', 997)


### Tokenization

In [4]:
# split the text and convert all words to lowercase
texts = textFilePairs.map(lambda (path, text):text.encode('utf8'))
totalWords = texts.flatMap(lambda text:map(lambda word:word.lower(), text.replace('\n', ' ').split()))
print(totalWords.distinct().count())
print(totalWords.take(100))

425542
['xref:', 'cantaloupe.srv.cs.cmu.edu', 'alt.atheism:49960', 'alt.atheism.moderated:713', 'news.answers:7054', 'alt.answers:126', 'path:', 'cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew', 'from:', 'mathew', '<mathew@mantis.co.uk>', 'newsgroups:', 'alt.atheism,alt.atheism.moderated,news.answers,alt.answers', 'subject:', 'alt.atheism', 'faq:', 'atheist', 'resources', 'summary:', 'books,', 'addresses,', 'music', '--', 'anything', 'related', 'to', 'atheism', 'keywords:', 'faq,', 'atheism,', 'books,', 'music,', 'fiction,', 'addresses,', 'contacts', 'message-id:', '<19930329115719@mantis.co.uk>', 'date:', 'mon,', '29', 'mar', '1993', '11:57:19', 'gmt', 'expires:', 'thu,', '29', 'apr', '1993', '11:57:19', 'gmt', 'followup-to:', 'alt.atheism', 'distribution:', 'world', 'organization:', 'mantis', 'consultants,', 'cambridge.', 'uk.',

In [5]:
# The preceding simple approach results in a lot of tokens and does not flter out many nonword characters 
#  We can do this by splitting each raw document on nonword characters using a regular expression pattern
import re
noPunctuationWords = texts.flatMap(lambda text:map(lambda word:word.lower(), re.split('\W+', text)))
print(noPunctuationWords.take(100))

['xref', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'alt', 'atheism', '49960', 'alt', 'atheism', 'moderated', '713', 'news', 'answers', '7054', 'alt', 'answers', '126', 'path', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'crabapple', 'srv', 'cs', 'cmu', 'edu', 'bb3', 'andrew', 'cmu', 'edu', 'news', 'sei', 'cmu', 'edu', 'cis', 'ohio', 'state', 'edu', 'magnus', 'acs', 'ohio', 'state', 'edu', 'usenet', 'ins', 'cwru', 'edu', 'agate', 'spool', 'mu', 'edu', 'uunet', 'pipex', 'ibmpcug', 'mantis', 'mathew', 'from', 'mathew', 'mathew', 'mantis', 'co', 'uk', 'newsgroups', 'alt', 'atheism', 'alt', 'atheism', 'moderated', 'news', 'answers', 'alt', 'answers', 'subject', 'alt', 'atheism', 'faq', 'atheist', 'resources', 'summary', 'books', 'addresses', 'music', 'anything', 'related', 'to', 'atheism', 'keywords', 'faq', 'atheism', 'books', 'music', 'fiction', 'addresses', 'contacts', 'message', 'id']


In [6]:
# filter out string with digits
onlyWords = noPunctuationWords.filter(lambda word: not re.search(r'\d', word))
print(onlyWords.take(100))

['xref', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'alt', 'atheism', 'alt', 'atheism', 'moderated', 'news', 'answers', 'alt', 'answers', 'path', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'crabapple', 'srv', 'cs', 'cmu', 'edu', 'andrew', 'cmu', 'edu', 'news', 'sei', 'cmu', 'edu', 'cis', 'ohio', 'state', 'edu', 'magnus', 'acs', 'ohio', 'state', 'edu', 'usenet', 'ins', 'cwru', 'edu', 'agate', 'spool', 'mu', 'edu', 'uunet', 'pipex', 'ibmpcug', 'mantis', 'mathew', 'from', 'mathew', 'mathew', 'mantis', 'co', 'uk', 'newsgroups', 'alt', 'atheism', 'alt', 'atheism', 'moderated', 'news', 'answers', 'alt', 'answers', 'subject', 'alt', 'atheism', 'faq', 'atheist', 'resources', 'summary', 'books', 'addresses', 'music', 'anything', 'related', 'to', 'atheism', 'keywords', 'faq', 'atheism', 'books', 'music', 'fiction', 'addresses', 'contacts', 'message', 'id', 'mantis', 'co', 'uk', 'date', 'mon']


### Remove stop words

We can take a look at some of the tokens in our corpus that have the highest occurrence 
across all documents to get an idea about some other stop words to exclude

In [7]:
wordCount = onlyWords.map(lambda word:(word, 1)).reduceByKey(lambda a,b : a+b)
sortedWordCount = wordCount.sortBy(lambda (k, v):v, ascending = False)
print(sortedWordCount.take(20))

[('the', 256555), ('edu', 164007), ('to', 133963), ('of', 122352), ('a', 111811), ('and', 102358), ('i', 92113), ('in', 87008), ('is', 75575), ('that', 70765), ('ax', 62416), ('it', 58816), ('cmu', 52409), ('for', 50392), ('com', 50158), ('you', 48181), ('cs', 45142), ('from', 39705), ('s', 38681), ('on', 35559)]


In [26]:
stopWords = {"the","a","s","an","of","or","in","for","by","on","but", "is", "not","with", "as", "was", "if",
             "they", "are", "this", "and", "it", "have", "from", "at", "my","be", "that", "to"}
print('before filtering stop words: {0}'.format(sortedWordCount.count()))
filteredSortedWordCount = sortedWordCount.filter(lambda (k, v): k not in stopWords)
print('after filtering stop words:{0}'.format(filteredSortedWordCount.count()))
print(filteredSortedWordCount.take(20))
rareWords = set(filteredSortedWordCount.filter(lambda (k,v): v==1).collect())
print(type(rareWords), len(rareWords))

before filtering stop words: 111525
after filtering stop words:111496
[('edu', 164007), ('i', 92113), ('ax', 62416), ('cmu', 52409), ('com', 50158), ('you', 48181), ('cs', 45142), ('news', 34309), ('srv', 32359), ('t', 32121), ('cantaloupe', 26048), ('net', 25459), ('message', 21954), ('subject', 21589), ('lines', 20894), ('date', 20787), ('id', 20695), ('apr', 20510), ('newsgroups', 20404), ('path', 20369)]
(<type 'set'>, 40808)


One other fltering step that we will use is removing any tokens that are only
one character in length. The reasoning behind this is similar to removing stop
words—these single-character tokens are unlikely to be informative in our text
model and can further reduce the feature dimension and model size

In [9]:
filteredSortedWordCount = filteredSortedWordCount.filter(lambda (k, v): len(k) > 1)
print(filteredSortedWordCount.count())
print(filteredSortedWordCount.take(20))

111470
[('edu', 164007), ('ax', 62416), ('cmu', 52409), ('com', 50158), ('you', 48181), ('cs', 45142), ('news', 34309), ('srv', 32359), ('cantaloupe', 26048), ('net', 25459), ('message', 21954), ('subject', 21589), ('lines', 20894), ('date', 20787), ('id', 20695), ('apr', 20510), ('newsgroups', 20404), ('path', 20369), ('can', 20028), ('organization', 19840)]


### Excluding terms based on frequency

It is also a common practice to exclude terms during tokenization when their overall
occurrence in the corpus is very low

In [24]:
print(filteredSortedWordCount.takeOrdered(20, lambda (k, v):v))

[('tilton', 2), ('netcdf', 2), ('outragious', 2), ('fawr', 2), ('sation', 2), ('yougoslavie', 2), ('gruel', 2), ('originality', 2), ('_ki', 2), ('xjudging', 2), ('lmx', 2), ('centimeter', 2), ('phenylanine', 2), ('wiseguy', 2), ('natured', 2), ('naviagtion', 2), ('vecchio', 2), ('sig_alrm', 2), ('toowoomba', 2), ('millimetres', 2)]
(<type 'list'>, 0)


As we can see, there are many terms that only occur once in the entire corpus.
Since typically we want to use our extracted features for other tasks such as
document similarity or machine learning models, tokens that only occur once are
not useful to learn from, as we will not have enough training data relative to these
tokens. We can apply another filter to exclude these rare tokens

In [11]:
print('before filtering words that appear once: {0}'.format(filteredSortedWordCount.count()))
filteredSortedWordCount = filteredSortedWordCount.filter(lambda (k, v) : v > 1)
print('after filtering words that appear once:{0}'.format(filteredSortedWordCount.count()))
print(filteredSortedWordCount.takeOrdered(20, lambda (k, v):v))

before filtering words that appear once: 111470
after filtering words that appear once:70662
[('tilton', 2), ('netcdf', 2), ('outragious', 2), ('fawr', 2), ('sation', 2), ('yougoslavie', 2), ('gruel', 2), ('originality', 2), ('_ki', 2), ('xjudging', 2), ('lmx', 2), ('centimeter', 2), ('phenylanine', 2), ('wiseguy', 2), ('natured', 2), ('naviagtion', 2), ('vecchio', 2), ('sig_alrm', 2), ('toowoomba', 2), ('millimetres', 2)]


In [31]:
# combine all the processing procedure above to  tranfrom each document to a sequence of tokens
tokens = texts.map(lambda text :map(lambda word:word.lower(), re.split('\W+', text)))\
              .map(lambda wordList: filter(lambda word: not re.search(r'\d', word), wordList))\
              .map(lambda wordList: filter(lambda word: word not in stopWords, wordList))\
              .map(lambda wordList: filter(lambda word: word not in rareWords, wordList))\
              .map(lambda wordList: filter(lambda word: len(word)>1, wordList))

### A note about stemming

A common step in text processing and tokenization is stemming. **This is the conversion
of whole words to a base form (called a word stem)**. For example, plurals might be
converted to singular (dogs becomes dog), and forms such as walking and walker might
become walk. Stemming can become quite complex and is typically handled with
specialized NLP or search engine software (such as NLTK, OpenNLP, and Lucene,
for example). We will ignore stemming for the purpose of our example here.

# Training a TF-IDF model

We will now use MLlib to transform each document, in the form of processed
tokens, into a vector representation. The frst step will be to **use the HashingTF
implementation, which makes use of feature hashing to map each token in the input
text to an index in the vector of term frequencies.** Then, we will compute the global
IDF and use it to transform the term frequency vectors into TF-IDF vectors

In [32]:
print(tokens.first())

['xref', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'alt', 'atheism', 'alt', 'atheism', 'moderated', 'news', 'answers', 'alt', 'answers', 'path', 'cantaloupe', 'srv', 'cs', 'cmu', 'edu', 'crabapple', 'srv', 'cs', 'cmu', 'edu', 'andrew', 'cmu', 'edu', 'news', 'sei', 'cmu', 'edu', 'cis', 'ohio', 'state', 'edu', 'magnus', 'acs', 'ohio', 'state', 'edu', 'usenet', 'ins', 'cwru', 'edu', 'agate', 'spool', 'mu', 'edu', 'uunet', 'pipex', 'ibmpcug', 'mantis', 'mathew', 'mathew', 'mathew', 'mantis', 'co', 'uk', 'newsgroups', 'alt', 'atheism', 'alt', 'atheism', 'moderated', 'news', 'answers', 'alt', 'answers', 'subject', 'alt', 'atheism', 'faq', 'atheist', 'resources', 'summary', 'books', 'addresses', 'music', 'anything', 'related', 'atheism', 'keywords', 'faq', 'atheism', 'books', 'music', 'fiction', 'addresses', 'contacts', 'message', 'id', 'mantis', 'co', 'uk', 'date', 'mon', 'mar', 'gmt', 'expires', 'thu', 'apr', 'gmt', 'followup', 'alt', 'atheism', 'distribution', 'world', 'organization', 'mant

In [34]:
from pyspark.mllib.feature import HashingTF, IDF
# ref: https://spark.apache.org/docs/2.1.1/mllib-feature-extraction.html
dim = pow(2, 18)

# The transform function of HashingTF maps each input document 
# that is, a sequence of tokens to an MLlib Vector.
hashingTF = HashingTF(dim)
tf = hashingTF.transform(tokens)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [45]:
TFIDFDoc = tfidf.first()
print(type(TFIDFDoc), TFIDFDoc.size, TFIDFDoc.values.size)
print(TFIDFDoc.values[:10])
print(TFIDFDoc.indices[:10])

(<class 'pyspark.mllib.linalg.SparseVector'>, 262144, 788)
[ 1.2445212   2.9383072   6.60755068  1.53191401  2.13460871  3.18437439
  2.71951683  5.61292811  6.53609172  5.54657807]
[ 180 1025 1542 1580 1595 2263 2424 2970 3407 3484]


We can see that the dimension of each sparse vector of term frequencies is 262144 (or 2^18 as we specifed).
However, the number on non-zero entries in the vector is only 788. 
The last two lines of the output show the tfidf and vector indexes for the first few entries in the vector

In [44]:
# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

## Analyzing the TF-IDF weightings

In [48]:
# First, we can compute the minimum and maximum TF-IDF weights across the entire corpus
minMaxTFIDF = tfidf.map(lambda tfidfDoc: (min(tfidfDoc.values), max(tfidfDoc.values)))\
                   .reduce(lambda (min1, max1), (min2, max2): (min(min1, min2), max(max1, max2)))
print(minMaxTFIDF)

(0.0, 71443.118724658925)


In [54]:
# TF-IDF weighting will tend to assign a lower weighting to common terms. 
# To see this, we can compute the TF-IDF representation for a few of the terms that appear in
# the list of top occurrences that we previously computed, such as you, do, and we
common = sc.parallelize(['you', 'do', 'we'])
tfCommon = hashingTF.transform(common)
tfidfCommon = idf.transform(tfCommon)
print(tfidfCommon.first().values)

[ 9.90338755  9.90338755  9.90338755]


In [56]:
# Now, let's apply the same transformation to a few less common terms that we might
# intuitively associate with being more linked to specifc topics or concepts
uncommon = sc.parallelize(['telescope', 'legislation', 'investment'])
tfUncommon = hashingTF.transform(uncommon)
tfidfUncommon = idf.transform(tfUncommon)
print(tfidfUncommon.first().values)

[  9.90338755   9.21024037   9.90338755  23.11848891   8.80477526
   9.90338755   8.51709319]


# Using a TF-IDF model

**While we often refer to training a TF-IDF model, it is actually a feature extraction process or transformation rather than a machine learning model. **TF-IDF weighting is often used as a preprocessing step for other models, such as dimensionality reduction, classifcation, or regression.

## Document similarity with the 20 Newsgroups dataset and TF-IDF features

## Training a text classifer on the 20 Newsgroups dataset using TF-IDF

When using TF-IDF vectors, we expected that the cosine similarity measure would
capture the similarity between documents, based on the overlap of terms between
them